In [1]:
import glob
import pandas as pd
import numpy as np
from collections import Counter

In [12]:
#below choose which directory to work with (all files or test files)

#path = 'homepage_csvs/test.chaturbate/' #change platform name after full stop to choose which test files to use
#path = 'homepage_csvs/chaturbate/'
#path = 'homepage_csvs/myfreecams/'
#path = 'homepage_csvs/bongacams/'
#path = 'homepage_csvs/livejasmin/'
#path = 'homepage_csvs/streamate/'
#path = 'homepage_csvs/ebonycams/'

#DEPENDING ON THE PLATFORM, MAKE SURE TO ALSO DEAL WITH FILE NAMES OF SAVED CSV LATER!

# get all filenames from the directory
csv_files = glob.glob(path + '*.csv')

## Collecting needed metrics from csv files
- this is code for reading multiple frontpage files and calculating metrics per timestamp
- code is based on the example from "basic_file_read" by Bernhard
- we collect metrics: for each day in the studied timeframe and also for each hour in 24hrs, not separating per date


In [13]:

daily_dict = {} #create an empty dictionary to store all metrics for daily calculations
daily_tags = {} #create an empty dictionary to store a list of all hashtags per day
hourly_dict = {} #make the master dictionary for collecting hourly data
#IF HASHTAGS ARE INTERESTING PER TIME IN DAY, THEN HERE THAT SHOULD BE ADDED THE SAME WAY AS IN METRICS PER DAY

counter = 1

# iterate over all files to create an array of lists with all metrics for each day
for filename in sorted(csv_files):

    # extract date from filename
    #print(filename)
    fileparts = filename.split('_')
    smaller_parts = fileparts[1].split('/')
    platform_split = smaller_parts[2].split('2')
    platform = platform_split[0]
    #print(platform)
    
    # myfreecams has no underscore after platform name, so the date/time extraction is different:
    if platform == "myfreecams":
        myfreecams_split = smaller_parts[2].split('myfreecams')
        date = myfreecams_split[1]
        time = fileparts[2]
        #print(time)
        #print(date)
    else: #for the rest of the platforms it's simple:
        date = fileparts[2]
        time = fileparts[3]
        #print(time)
        #print(date)


    timeparts = time.split('-')
    hour = f"0000-00-00 {timeparts[0]}:{timeparts[1]}:00" #the weird format is because i need that for some visualizations
    #print(hour)
    
    print(f"{counter} platform:{platform} date:{date} time:{hour}")
    counter += 1

    if date not in daily_dict: #add the date to a dictionary with keys for each date
        daily_dict[date] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each date for metrics and list for each value
        daily_tags[date] = [] #daily also creating hashtags, but not by hour

    
    if hour not in hourly_dict: #add the hour to a dictionary with keys for each hour
        hourly_dict[hour] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each halfhour for metrics and list for each value


    # __________________________read CSV file and get metrics________________________________________

    df = pd.read_csv(filename) #open the csv as dataframe
    files = 1
    
    all_performers_count = len(df)
    
    if "offline" in df.columns:
        offline_count = df.offline.sum()
        df = df[df["offline"] == False] #filtering out the rows that have offline performers
        #print("offline performers might exist (ebony), but were removed")
    elif "away" in df.columns:
        offline_count = df.away.sum()
        df = df[df["away"] == False] #filtering out the rows that have offline performers
        #print("offline performers might existe (bonga), but were removed")
    else:
        offline_count = 0
        #print("this plaform does not save offline performers, proceeding >>")
    
    
    perf_count = len(df) #number of shows in each csv file
    print(f"number of online performers:{perf_count}, number of offline performers: {offline_count}, number of all performers: {all_performers_count}")
    print(f"{perf_count}?")
    print(all_performers_count - offline_count) #check if i did not fuck up by checking if the numbers are the same
    
# _______________viewers___________

    if "viewers" in df.columns:
        df['viewers'] = df['viewers'].astype(str) #first make all viewer counts into string
        df['viewers'] = df['viewers'].str.replace('+', '') #then remove any "+"" from strings
        df["viewers"] = pd.to_numeric(df["viewers"]) #then make them into integers again -- this is to fix bongacams issue of having "99999+" as a number of viewers
        #df["viewers"] = df["viewers"].astype(int)
        view_sum = df["viewers"].sum() #sum of viewers in each csv file
        #print(f"got sum of viewers chaturbate/bongacams: {view_sum}")
    elif "room_count" in df.columns:
        view_sum = df["room_count"].sum()
        #print(f"got sum of viewers myfreecams: {view_sum}")    
    else:
        view_sum = 0
        #print("no viewer numbers for this platform")
    
    #try:
    #    show_len_sum = sum(df["time"]) #sum of show time in each csv file
    #except:
    #    show_len_sum = 0
    #    print("this platform does not provide show lengths")
    #skipping this part because it is only on chaturbate and is not accurate, because it is not actually show length, 
    #but how long the performer has been online at the moment of scrape
    
# _______________gender, couples___________
    #this is only for chaturbate:
    if "female" in df.columns:    #they all go together
        female_count = df.female.sum() #number of female performers in chaturbate files
        male_count = df.male.sum() #number of male performers
        trans_count = df.trans.sum() #number of trans performers
        couple_count = df.couple.sum() #number of couples performing

    else:
        female_count = 0
        male_count = 0
        trans_count = 0
        couple_count = 0
        #print("this platform does not provide gender indications")

# _______________vibrator use___________
    if "vibrator" in df.columns:
        vibrator_count = df.vibrator.sum() #number of performances with smart vibrators activated 
    else:
        vibrator_count = 0
        #print("this platform does not provide vibrator indication")

# _______________new performers___________
    if "new" in df.columns: 
        new_count = df.new.sum() #number of new performers 
    else:
        new_count = 0
        #print("this platform does not provide new performer indication")
    
# _______________performers in private shows___________   
    if "private" in df.columns:
        private_count = df.private.sum()  #number of performers in a private show at the moment (bongacams)
        #print(f"got bongacams privates: {private_count}")
    elif "private_show" in df.columns:
        private_count = df.private_show.sum() + df.true_private_show.sum() #number of performers in a private show at the moment (myfreecams)
        #print(f"got myfreecams privates: {private_count}")
    else:
        private_count = 0
        #print("this platform does not provide private show indication")
    
# _______________promoted performers___________    

    if "promoted" in df.columns:
        promoted_count = df.promoted.sum()
        #print(f"got promoted performers (livejasmin/chaturbate): {promoted_count}")
    elif "lifted_up_webcam_model" in df.columns:
        promoted_count = df.lifted_up_webcam_model.sum()
        #print(f"got promoted performers (bongacams): {promoted_count}")
    else:
        promoted_count = 0
        #print("this platform does not provide promoted performer indication")

# _______________performers in group shows___________   
    if "group" in df.columns:
        group_count = df.group.sum() #bonga
        #print(f"got performers in group shows (bongacams): {group_count}")
    elif "group_show" in df.columns:
        group_count = df.group_show.sum() #myfreecams
        #print(f"got performers in group shows (myfreecams): {group_count}")
    else:
        group_count = 0
        #print("this platform does not provide group show indication")


   # __________________________put metrics in a dictionary________________________________________
    # add all the new collected values in the nested dictionary for each day
    daily_dict[date]["perf_counts"].append(perf_count)
    daily_dict[date]["view_sums"].append(view_sum)
    daily_dict[date]["female_counts"].append(female_count)
    daily_dict[date]["male_counts"].append(male_count)
    daily_dict[date]["trans_counts"].append(trans_count)
    daily_dict[date]["couple_counts"].append(couple_count)
    daily_dict[date]["vibrator_counts"].append(vibrator_count)
    daily_dict[date]["new_counts"].append(new_count)
    daily_dict[date]["private_counts"].append(private_count)
    daily_dict[date]["offline_counts"].append(offline_count)
    daily_dict[date]["promoted_counts"].append(promoted_count)
    daily_dict[date]["group_counts"].append(group_count)
    daily_dict[date]["number_files"].append(files)
    print("collected daily metrics")

    # add all the new collected values in the nested dictionary for each hour in 24hrs
    hourly_dict[hour]["perf_counts"].append(perf_count)
    hourly_dict[hour]["view_sums"].append(view_sum)
    hourly_dict[hour]["female_counts"].append(female_count)
    hourly_dict[hour]["male_counts"].append(male_count)
    hourly_dict[hour]["trans_counts"].append(trans_count)
    hourly_dict[hour]["couple_counts"].append(couple_count)
    hourly_dict[hour]["vibrator_counts"].append(vibrator_count)
    hourly_dict[hour]["new_counts"].append(new_count)
    hourly_dict[hour]["private_counts"].append(private_count)
    hourly_dict[hour]["offline_counts"].append(offline_count)
    hourly_dict[hour]["promoted_counts"].append(promoted_count)
    hourly_dict[hour]["group_counts"].append(group_count)
    hourly_dict[hour]["number_files"].append(files)
    print("collected hourly metrics")



# __________________________collect hashtags per day and put in dictionary________________________________________

    if "tags" in df.columns:
        hashtags_list = df["tags"].tolist() #makes a list of all the hashtags used that day
        
        hashtags = []
        for object in hashtags_list:
            #print(object)
            if type(object) is str:
                #print(type(object))
                object = object.split(",")
                for word in object:
                    hashtags.append(word)
            else:
                #print(type(object))
                hashtags.append(object)
        #print(hashtags)

    #add the collected hashtags in their own dictionary
        daily_tags[date]+=hashtags
        print("collected daily hashtags")
    else:
        hashtags_list = 0
        print("this platform does not provide hashtags")


#print(daily_dict)
#print(hourly_dict)
#print(daily_tags)

1 platform:ebonycams date:2021-11-03 time:0000-00-00 20:00:00
number of online performers:1152, number of offline performers: 0, number of all performers: 1152
1152?
1152
collected daily metrics
collected hourly metrics
this platform does not provide hashtags
2 platform:ebonycams date:2021-11-03 time:0000-00-00 20:30:00
number of online performers:2105, number of offline performers: 7399, number of all performers: 9504
2105?
2105
collected daily metrics
collected hourly metrics
this platform does not provide hashtags
3 platform:ebonycams date:2021-11-03 time:0000-00-00 21:00:00
number of online performers:2317, number of offline performers: 7187, number of all performers: 9504
2317?
2317
collected daily metrics
collected hourly metrics
this platform does not provide hashtags
4 platform:ebonycams date:2021-11-03 time:0000-00-00 21:30:00
number of online performers:2400, number of offline performers: 6960, number of all performers: 9360
2400?
2400
collected daily metrics
collected hourly

## Making daily metrics output file with calculations

In [14]:
daily_avgs = []

# next step is to iterate over array to calculate daily averages and counts

for date in daily_dict:  #for each date calculate:
    daily_files_sum = np.sum(daily_dict[date]["number_files"]) #how many files collected for that day
    daily_shows_avg = np.mean(daily_dict[date]["perf_counts"]) #average length of page (number of performances - the ones that are online) (all platforms)
    daily_offline_avg = np.mean(daily_dict[date]["offline_counts"]) #average of how many people on the list were offline (bongacams and myfreecams)
    daily_views_sum = sum(daily_dict[date]["view_sums"]) #sum of all viewers that day (chaturbate, bongacams and myfreecams)
    daily_views_sum_avg = daily_views_sum / daily_files_sum #added this now -- average of total viewers on the platform that day (chaturbate, bongacams and myfreecams)
    daily_views_avg = daily_views_sum / sum(daily_dict[date]["perf_counts"]) #average number of viewers in a show that day (chaturbate, bongacams and myfreecams)
    
    daily_female_avg = np.mean(daily_dict[date]["female_counts"]) #only chaturbate
    daily_female_percent = sum(daily_dict[date]["female_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_male_avg = np.mean(daily_dict[date]["male_counts"]) #only chaturbate
    daily_male_percent = sum(daily_dict[date]["male_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_trans_avg = np.mean(daily_dict[date]["trans_counts"]) #only chaturbate
    daily_trans_percent = sum(daily_dict[date]["trans_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_couple_avg = np.mean(daily_dict[date]["couple_counts"]) #only chaturbate
    daily_couple_percent = sum(daily_dict[date]["couple_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    
    daily_vibrator_avg = np.mean(daily_dict[date]["vibrator_counts"]) #(bongacams and livejasmin)
    daily_vibrator_percent = sum(daily_dict[date]["vibrator_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #(bongacams and livejasmin)
    daily_new_avg = np.mean(daily_dict[date]["new_counts"]) #bongacams, chaturbate, myfreecams, livejasmin
    daily_new_percent = sum(daily_dict[date]["new_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams, chaturbate, myfreecams, livejasmin
    daily_promoted_avg = np.mean(daily_dict[date]["promoted_counts"]) #bongacams, livejasmin, chaturbate
    daily_promoted_percent = sum(daily_dict[date]["promoted_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams, livejasmin, chaturbate

    daily_private_avg = np.mean(daily_dict[date]["private_counts"]) #bongacams and myfreecams
    daily_private_percent = sum(daily_dict[date]["private_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams and myfreecams
    daily_group_avg = np.mean(daily_dict[date]["group_counts"]) #bongacams and myfreecams
    daily_group_percent = sum(daily_dict[date]["group_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams and myfreecams


    #then add everything to a nice dictionary
    daily_avgs.append({'date':date, 'number_files': daily_files_sum, 'shows_average':round(daily_shows_avg, 2), 'offline_average':round(daily_offline_avg, 2), 
    'viewers_sum':daily_views_sum , 'viewers_sum_avg':round(daily_views_sum_avg, 2), 'viewers_average_in_show':round(daily_views_avg, 2),
    'average_vibrator':round(daily_vibrator_avg,2), 'percentage_vibrator':round(daily_vibrator_percent,2), 'average_new':round(daily_new_avg,2), 'percentage_new':round(daily_new_percent,2),
    'average_promoted':round(daily_promoted_avg,2), 'percentage_promoted':round(daily_promoted_percent,2), 'average_private':round(daily_private_avg,2), 'percentage_private':round(daily_private_percent,2),
    'average_group':round(daily_group_avg,2), 'percentage_group':round(daily_group_percent,2), 'average_females':round(daily_female_avg,2), 'percentage_females':round(daily_female_percent,2), 
    'average_males':round(daily_male_avg,2), 'percentage_males':round(daily_male_percent,2), 'average_trans':round(daily_trans_avg,2), 'percentage_trans':round(daily_trans_percent,2), 
    'average_couples':round(daily_couple_avg), 'percentage_couples':round(daily_couple_percent,2)})
    #print(daily_avgs)

#and make a dataframe from the dictionary
df = pd.DataFrame.from_dict(daily_avgs)
#df.to_csv('daily_metrics_ebonycams_22september.csv',index=False) #and save it as a csv file
df

,date,number_files,shows_average,offline_average,viewers_sum,viewers_sum_avg,viewers_average_in_show,average_vibrator,percentage_vibrator,average_new,...,average_group,percentage_group,average_females,percentage_females,average_males,percentage_males,average_trans,percentage_trans,average_couples,percentage_couples
0,2021-11-03,8,2153.75,5310.25,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,2021-11-04,48,2149.73,6869.27,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,2021-11-05,48,2210.77,7025.23,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,2021-11-06,48,2049.98,6677.02,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,2021-11-07,48,1564.12,7556.88,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2022-01-12,48,1972.58,7297.42,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
63,2022-01-13,48,2107.10,7314.90,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
64,2022-01-14,48,2039.92,6853.08,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
65,2022-01-15,48,2012.75,6846.25,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [7]:
# here i make a file with the counted hashtags per day
# FUTURE: not include all hashtags, but only those that are counted above a threshold?

daily_tags_counted = []

for date in daily_tags:  #iterating though every date
    daily_counts = Counter(daily_tags[date]) #instead of having a list of all hashtags, this makes a dictionary with counts of each hashtag
    #print(daily_counts)

    for tag in daily_counts: 
        daily_tags_counted.append({"date": date, "hashtag": tag , "count": daily_counts[tag]})
        #daily_tags_counted[date][tag] = daily_counts[tag]

df = pd.DataFrame.from_dict(daily_tags_counted)
#df.to_csv('daily_hashtags_myfreecams.csv',index=False)
df
    

,date,hashtag,count
0,2021-11-09,NaN,50177
1,2021-11-09,#lovense,41098
2,2021-11-09,#tattoo,3633
3,2021-11-09,#ink,28
4,2021-11-09,#latina,47531
...,...,...,...
5416,2021-11-12,#fountainsquirt,1
5417,2021-11-12,#bwc,1
5418,2021-11-12,#teasing,1
5419,2021-11-12,#africa,1


## Making halfhourly metrics output file with calculations

In [15]:
hourly_avgs = []

# next step is to iterate over array to calculate hourly metrics
for hour in hourly_dict:  #for each hour in day calculate:
    hourly_files_sum = np.sum(hourly_dict[hour]["number_files"]) #how many files collected for that halfhour interval
    hourly_shows_avg = np.mean(hourly_dict[hour]["perf_counts"]) #average length of page (number of performances - the ones that are online) (all platforms)
    hourly_offline_avg = np.mean(hourly_dict[hour]["offline_counts"]) #average of how many people on the list were offline (bongacams and myfreecams)
    hourly_views_sum = sum(hourly_dict[hour]["view_sums"]) #sum of all viewers on the platform over all collected data for that halfhour period (chaturbate, bongacams and myfreecams)
    hourly_views_sum_avg = hourly_views_sum / hourly_files_sum #average number of viewers on the platform (chaturbate, bongacams and myfreecams)
    hourly_views_avg_show =  hourly_views_sum / sum(hourly_dict[hour]["perf_counts"]) #average number of viewers in a show per halfhour (chaturbate, bongacams and myfreecams)

    hourly_female_avg = np.mean(hourly_dict[hour]["female_counts"]) #only chaturbate
    hourly_female_percent = sum(hourly_dict[hour]["female_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_male_avg = np.mean(hourly_dict[hour]["male_counts"]) #only chaturbate
    hourly_male_percent = sum(hourly_dict[hour]["male_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_trans_avg = np.mean(hourly_dict[hour]["trans_counts"]) #only chaturbate
    hourly_trans_percent = sum(hourly_dict[hour]["trans_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_couple_avg = np.mean(hourly_dict[hour]["couple_counts"]) #only chaturbate
    hourly_couple_percent = sum(hourly_dict[hour]["couple_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    
    hourly_vibrator_avg = np.mean(hourly_dict[hour]["vibrator_counts"]) #(bongacams and livejasmin)
    hourly_vibrator_percent = sum(hourly_dict[hour]["vibrator_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #(bongacams and livejasmin)
    hourly_new_avg = np.mean(hourly_dict[hour]["new_counts"]) #bongacams, chaturbate, myfreecams, livejasmin
    hourly_new_percent = sum(hourly_dict[hour]["new_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams, chaturbate, myfreecams, livejasmin
    hourly_promoted_avg = np.mean(hourly_dict[hour]["promoted_counts"]) #bongacams, livejasmin, chaturbate
    hourly_promoted_percent = sum(hourly_dict[hour]["promoted_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams, livejasmin, chaturbate

    hourly_private_avg = np.mean(hourly_dict[hour]["private_counts"]) #bongacams and myfreecams
    hourly_private_percent = sum(hourly_dict[hour]["private_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams and myfreecams
    hourly_group_avg = np.mean(hourly_dict[hour]["group_counts"]) #bongacams and myfreecams
    hourly_group_percent = sum(hourly_dict[hour]["group_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams and myfreecams


    #then add everything to a nice dictionary
    hourly_avgs.append({'hour':hour, 'number_files': hourly_files_sum, 'shows_average':round(hourly_shows_avg, 2), 'offline_average':round(hourly_offline_avg, 2),
    'viewers_sum':hourly_views_sum, 'viewers_average_total':round(hourly_views_sum_avg,2), 'viewers_average_in_show':round(hourly_views_avg_show, 2),
    'average_vibrator':round(hourly_vibrator_avg,2), 'percentage_vibrator':round(hourly_vibrator_percent,2), 'average_new':round(hourly_new_avg,2), 'percentage_new':round(hourly_new_percent,2),
    'average_promoted':round(hourly_promoted_avg,2), 'percentage_promoted':round(hourly_promoted_percent,2), 'average_private':round(hourly_private_avg,2), 'percentage_private':round(hourly_private_percent,2),
    'average_group':round(hourly_group_avg,2), 'percentage_group':round(hourly_group_percent,2), 'average_females':round(hourly_female_avg,2), 'percentage_females':round(hourly_female_percent,2), 
    'average_males':round(hourly_male_avg,2), 'percentage_males':round(hourly_male_percent,2), 'average_trans':round(hourly_trans_avg,2), 'percentage_trans':round(hourly_trans_percent,2), 
    'average_couples':round(hourly_couple_avg), 'percentage_couples':round(hourly_couple_percent,2)})
    #print(hourly_avgs)

#and make a dataframe from the dictionary
df = pd.DataFrame.from_dict(hourly_avgs)
#df.to_csv('hourly_metrics_ebonycams_22september.csv',index=False) #and save it as a csv file
df

,hour,number_files,shows_average,offline_average,viewers_sum,viewers_average_total,viewers_average_in_show,average_vibrator,percentage_vibrator,average_new,...,average_group,percentage_group,average_females,percentage_females,average_males,percentage_males,average_trans,percentage_trans,average_couples,percentage_couples
0,0000-00-00 20:00:00,63,1636.24,7459.38,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,0000-00-00 20:30:00,63,1833.84,7805.78,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,0000-00-00 21:00:00,62,1847.58,6893.06,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,0000-00-00 21:30:00,63,2039.08,6831.78,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,0000-00-00 22:00:00,64,2101.97,6950.53,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
5,0000-00-00 22:30:00,64,2105.30,6878.20,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
6,0000-00-00 23:00:00,65,2030.15,6176.37,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
7,0000-00-00 23:30:00,64,2063.12,7043.38,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
8,0000-00-00 00:00:00,64,1967.48,6822.52,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9,0000-00-00 00:30:00,64,1904.28,7262.86,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


## Checking for missing data

In [72]:
#first starting with collecting lists of dates that don't contain enough data (per platform)

missing_df = pd.read_csv("missing_data_overview.csv") #open the spreadcheet with data collection overview
all_added_df = missing_df.groupby(["date"]).sum() #groupby the dataframe by date and add up everything in all other columns (we are only using the ones with True/False and they add up each True as 1 )

#below i make a list of all dates that have missing data for any one of the platforms:
#first, indicate the rows in the dataframe
full_missing_data_df = all_added_df.loc[(all_added_df["contains_bongacams"] < 47) | (all_added_df["contains_chaturbate"] < 47) | (all_added_df["contains_livejasmin"] < 47) | (all_added_df["contains_myfreecams"] < 47) | (all_added_df["contains_streamate"] < 47) | (all_added_df["contains_ebonycams"] < 47)]
#then make a list of all the dates that have missing data
full_missing_data_lst = full_missing_data_df.index.tolist
#this, however, leads to a lot of dates being dismissed because one of the platforms has some missing data -- most of the time the analysis I do will probably analyze platforms one by one
#so it makes sense to make separate lists for each platform

#create a function that makes a list for each platform:
def getgapdateslist(df, platform, ok_to_miss):
    print(platform)
    params = dict(df = "df", platform="platform", ok_to_miss = "ok_to_miss") #df = what dataframe to use to get data from; platform = platform name, not capitalized, ok_to_miss = how many data files during the day are okay to miss?
    missing_data_df = df.loc[(df["contains_"+platform] < (48-ok_to_miss)) ]
    missing_data_lst = missing_data_df.index.tolist
    print(missing_data_lst)
    return missing_data_lst


#run the function and create a list for each platform
#i used the grouped full dataframe from above and allow one data file to be missing in 24hrs, so instead of 48 having 47 is ok

platform_names_lst = ["bongacams", "chaturbate", "streamate", "ebonycams", "livejasmin", "myfreecams"]

missing_data_dict = {}
for name in platform_names_lst:
    missing_data_dict[name] = getgapdateslist(all_added_df, name, 1)

print(missing_data_dict)

bongacams
<bound method IndexOpsMixin.tolist of Index(['2021-11-03', '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-24',
       '2021-11-28', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-07'],
      dtype='object', name='date')>
chaturbate
<bound method IndexOpsMixin.tolist of Index(['2021-11-03', '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-24',
       '2021-11-28', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-07',
       '2021-12-09', '2021-12-10', '2021-12-17', '2021-12-22', '2021-12-24',
       '2021-12-25', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-10',
       '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-16'],
      dtype='object', name='date')>
streamate
<bound method IndexOpsMixin.tolist of Index(['2021-11-03', '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-24',
       '2021-11-28', '2021-12-01', '2021-12-02', '2021-12-03', '2021-12-07',
       '2022-01-06'],
      dtype='object', name='date')>
ebonycams
<bound method IndexO

In [75]:
#another way to check for bias in missing data is to check how many times it is missing per halfhour
# and then compare if the difference is statistically significant
import scipy.stats as stats #importing the library that has "one sample t test"

added_time_df = missing_df.groupby(["time"]).sum() #groupby the dataframe by time and add up everything in all other columns (we are only using the ones with True/False and they add up each True as 1 )
#bonga = added_time_df["contains_bongacams"].to_list()
#bonga_max = max(bonga)

#create a function that checks for statistically significant difference between the mean of how many times a platform was scraped per each halfhour and the maximum number of times that platform was scraped
def checkifbiased(df, platform):
    print(platform)
    params = dict(df = "df", platform="platform", ok_to_miss = "ok_to_miss") #df = what dataframe to use to get data from; platform = platform name, not capitalized
    captured_lst = df["contains_"+platform].to_list() #gets all values in a list for that platform
    max_value = max(captured_lst) #gets max value from the list
    result = stats.ttest_1samp(a=captured_lst, popmean=max_value) #conducts one sample t test
    print(result) 
    return result
   

for name in platform_names_lst: #iterates through platform name list
    checkifbiased(added_time_df, name) #checks for each and prints results, when p value is higher than chosen alpha (e.g. 0.05), 
                                       #then there is no statistically significant difference



bongacams
Ttest_1sampResult(statistic=-14.679596930356633, pvalue=3.5451829855048786e-19)
chaturbate
Ttest_1sampResult(statistic=-13.581230831211249, pvalue=6.9215395838926815e-18)
streamate
Ttest_1sampResult(statistic=-11.081807235866505, pvalue=1.0495253313100673e-14)
ebonycams
Ttest_1sampResult(statistic=-11.054410884348407, pvalue=1.1423251062233537e-14)
livejasmin
Ttest_1sampResult(statistic=-15.332851801928792, pvalue=6.476813946219355e-20)
myfreecams
Ttest_1sampResult(statistic=-10.56844431357449, pvalue=5.2173139939145695e-14)



## -- IGNORE THIS --
Old version 

Bellow: collecting metrics separately per half an hour

In [ ]:
#make the master dictionary for collecting hourly data
hourly_dict = {}
#IF HASHTAGS ARE INTERESTING PER TIME IN DAY, THEN HERE THAT SHOULD BE ADDED THE SAME WAY AS IN METRICS PER DAY
counter = 1


# iterate over all files to create an array of lists with all metrics for each hour in a day
for filename in sorted(csv_files):

    #print(filename)
    fileparts = filename.split('_')
    smaller_parts = fileparts[1].split('/')
    platform_split = smaller_parts[2].split('2')
    platform = platform_split[0]
    #print(platform)
    
    if platform == "myfreecams":
        time = fileparts[2]
        print(time)
    else:
        time = fileparts[3]
        print(time)
   
    timeparts = time.split('-')
    hour = f"0000-00-00 {timeparts[0]}:{timeparts[1]}:00"
    #print(hour)
    
    print(str(counter) + ' ' + hour + ' ' + platform)
    counter += 1


    if hour not in hourly_dict: #add the hour to a dictionary with keys for each hour
        hourly_dict[hour] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each date for metrics and list for each value


    # __________________________read CSV file and get metrics________________________________________
    # this obviously could be done together with hourly metrics, so the files don't need to be opened twice, but for now i am keeping it this way
    # because i already have run the metrics for days in the timeframe

    df = pd.read_csv(filename) #open the csv as dataframe
    files = 1
    
    all_performers_count = len(df)
    
    if "offline" in df.columns:
        offline_count = df.offline.sum()
        df = df[df["offline"] == False] #filtering out the rows that have offline performers
        #print("offline performers might exist (ebony), but were removed")
    elif "away" in df.columns:
        offline_count = df.away.sum()
        df = df[df["away"] == False] #filtering out the rows that have offline performers
        #print("offline performers might existe (bonga), but were removed")
    else:
        offline_count = 0
        #print("this plaform does not save offline performers, proceeding >>")
    
    
    perf_count = len(df) #number of shows in each csv file
    print(f"number of online performers:{perf_count}, number of offline performers: {offline_count}, number of all performers: {all_performers_count}")
    print(f"{perf_count}?")
    print(all_performers_count - offline_count)
    
# _______________viewers___________

    if "viewers" in df.columns:
        df['viewers'] = df['viewers'].astype(str) #first make all viewer counts into string
        df['viewers'] = df['viewers'].str.replace('+', '') #then remove any "+"" from strings
        df["viewers"] = pd.to_numeric(df["viewers"]) #then make them into integers again -- this is to fix bongacams issue of having "99999+" as a number of viewers
        #df["viewers"] = df["viewers"].astype(int)
        view_sum = df["viewers"].sum() #sum of viewers in each csv file
        print(f"got sum of viewers chaturbate/bongacams: {view_sum}")
    elif "room_count" in df.columns:
        view_sum = df["room_count"].sum()
        print(f"got sum of viewers myfreecams: {view_sum}")    
    else:
        view_sum = 0
        print("no viewer numbers for this platform")
    
    #try:
    #    show_len_sum = sum(df["time"]) #sum of show time in each csv file
    #except:
    #    show_len_sum = 0
    #    print("this platform does not provide show lengths")
    #skipping this part because it is only on chaturbate and is not accurate, because it is not actually show length, 
    #but how long the performer has been online at the moment of scrape
    
# _______________gender, couples___________
    #this is only for chaturbate:
    if "female" in df.columns:    #they all go together
        female_count = df.female.sum() #number of female performers in chaturbate files
        male_count = df.male.sum() #number of male performers
        trans_count = df.trans.sum() #number of trans performers
        couple_count = df.couple.sum() #number of couples performing

    else:
        female_count = 0
        male_count = 0
        trans_count = 0
        couple_count = 0
        #print("this platform does not provide gender indications")

# _______________vibrator use___________
    if "vibrator" in df.columns:
        vibrator_count = df.vibrator.sum() #number of performances with smart vibrators activated 
    else:
        vibrator_count = 0
        #print("this platform does not provide vibrator indication")

# _______________new performers___________
    if "new" in df.columns: 
        new_count = df.new.sum() #number of new performers 
    else:
        new_count = 0
        #print("this platform does not provide new performer indication")
    
# _______________performers in private shows___________   
    if "private" in df.columns:
        private_count = df.private.sum()  #number of performers in a private show at the moment (bongacams)
        #print(f"got bongacams privates: {private_count}")
    elif "private_show" in df.columns:
        private_count = df.private_show.sum() + df.true_private_show.sum() #number of performers in a private show at the moment (myfreecams)
        #print(f"got myfreecams privates: {private_count}")
    else:
        private_count = 0
        #print("this platform does not provide private show indication")
    
# _______________promoted performers___________    

    if "promoted" in df.columns:
        promoted_count = df.promoted.sum()
        #print(f"got promoted performers (livejasmin/chaturbate): {promoted_count}")
    elif "lifted_up_webcam_model" in df.columns:
        promoted_count = df.lifted_up_webcam_model.sum()
        #print(f"got promoted performers (bongacams): {promoted_count}")
    else:
        promoted_count = 0
        #print("this platform does not provide promoted performer indication")

# _______________performers in group shows___________   
    if "group" in df.columns:
        group_count = df.group.sum() #bonga
        #print(f"got performers in group shows (bongacams): {group_count}")
    elif "group_show" in df.columns:
        group_count = df.group_show.sum() #myfreecams
        #print(f"got performers in group shows (myfreecams): {group_count}")
    else:
        group_count = 0
        #print("this platform does not provide group show indication")


   # __________________________put metrics in a dictionary________________________________________
    # add all the new collected values in the nested dictionary
    hourly_dict[hour]["perf_counts"].append(perf_count)
    hourly_dict[hour]["view_sums"].append(view_sum)
    hourly_dict[hour]["female_counts"].append(female_count)
    hourly_dict[hour]["male_counts"].append(male_count)
    hourly_dict[hour]["trans_counts"].append(trans_count)
    hourly_dict[hour]["couple_counts"].append(couple_count)
    hourly_dict[hour]["vibrator_counts"].append(vibrator_count)
    hourly_dict[hour]["new_counts"].append(new_count)
    hourly_dict[hour]["private_counts"].append(private_count)
    hourly_dict[hour]["offline_counts"].append(offline_count)
    hourly_dict[hour]["promoted_counts"].append(promoted_count)
    hourly_dict[hour]["group_counts"].append(group_count)
    hourly_dict[hour]["number_files"].append(files)
